# Information
## How to
1. Set the parameters. **UPPERCASE** letters are user input variables
2. Run the reprojection cell


# Requirements

In [1]:
import os
from importlib_resources import files
from pathlib import Path

from beak.utilities import transformation


# Scaling

## Standard scaler

**Scaling** all numerical folders within a specified model configuration.<br>
Reads the <code>ROOT_FOLDER</code> and takes the <code>NUMERICAL</code> subfolder within each model configuration.

**User inputs**

In [2]:
BASE_PATH = files("beak.data")

BASE_NAME = Path("MCCAFFERTY23") / "PROCESSED"
BASE_SPATIAL = "EPSG_32615_RES_50_0"
METHODS = ["standard"]


**Select** subfolders to be scaled.

In [4]:
root_folder = BASE_PATH / BASE_NAME / BASE_SPATIAL

for folder in os.listdir(root_folder):
  if os.path.isdir(os.path.join(root_folder, folder)):
    print(folder)

GEOPHYSICS


In [5]:
SELECTION = ["GEOPHYSICS"]
input_folders = [root_folder / folder for folder in SELECTION]

print("Selected folders:")
for folder in input_folders:
  print(folder)

Selected folders:
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\MCCAFFERTY23\PROCESSED\EPSG_32615_RES_50_0\GEOPHYSICS


**Run Scaling**

In [6]:
for folder in input_folders:
  folder_relative = os.path.relpath(folder, root_folder)
  for method in METHODS:
    if method == "standard":
      method_name = "STD"
    if method == "minmax":
      method_name = "MINMAX"
      
    print(f"\nProcessing {folder} with {method} method...")
    output_folder = root_folder / str(folder_relative + "_SCALED_" + str.upper(method_name))
    transformation.scale_raster(folder, output_folder, method)


Processing s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\MCCAFFERTY23\PROCESSED\EPSG_32615_RES_50_0\GEOPHYSICS with standard method...
Selected folder: S:\Projekte\20230082_DARPA_CriticalMAAS_TA3\Bearbeitung\GitHub\beak-ta3\src\beak\data\MCCAFFERTY23\PROCESSED\EPSG_32615_RES_50_0\GEOPHYSICS
Total of folders found: 1
Files loaded: 16


Processing files: 100%|██████████| 16/16 [00:23<00:00,  1.47s/it]


## Log scaler

In [11]:
# Get file list
from beak.utilities.io import create_file_list

# Get file list from selected folder
file_list = create_file_list(input_folders[0])

print("File list:")
for file in file_list:
  print(file)

File list:
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\MCCAFFERTY23\PROCESSED\EPSG_32615_RES_50_0\GEOPHYSICS\Gravity.tif
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\MCCAFFERTY23\PROCESSED\EPSG_32615_RES_50_0\GEOPHYSICS\Gravity_HGM.tif
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\MCCAFFERTY23\PROCESSED\EPSG_32615_RES_50_0\GEOPHYSICS\Gravity_Up30km.tif
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\MCCAFFERTY23\PROCESSED\EPSG_32615_RES_50_0\GEOPHYSICS\Gravity_Up30km_HGM.tif
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\MCCAFFERTY23\PROCESSED\EPSG_32615_RES_50_0\GEOPHYSICS\LAB.tif
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\MCCAFFERTY23\PROCESSED\EPSG_32615_RES_50_0\GEOPHYSICS\LAB_HGM_cog.tif
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung

In [12]:
# Choose files
convert_to_log = []

for file in file_list:
  if file.stem == "Mag_AnalyticSignal_cog":
    convert_to_log.append(file)

print("Files to convert to log:")
for file in convert_to_log:
  print(file)


Files to convert to log:
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\MCCAFFERTY23\PROCESSED\EPSG_32615_RES_50_0\GEOPHYSICS\Mag_AnalyticSignal_cog.tif


In [26]:
import rasterio
import numpy as np

from beak.utilities.io import check_path, save_raster

# Convert to log
for file in convert_to_log:
  print(f"\nConverting {file} to log...")
  file = Path(file)
  OUT_FOLDER = str(file.parent) + "_SCALED_LOG"
  OUT_FOLDER = check_path(OUT_FOLDER)
  
  OUT_PATH = Path(OUT_FOLDER) / file.name
  
  raster = rasterio.open(file)
  raster_meta = raster.meta.copy()
  
  data = raster.read(1)
  data = np.where(data == raster.nodata, np.nan, data)
  data = np.log(data)
  data = np.where(np.isnan(data), raster.nodata, data)
  
  save_raster(OUT_PATH, data, metadata=raster_meta)
  


Converting s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\MCCAFFERTY23\PROCESSED\EPSG_32615_RES_50_0\GEOPHYSICS\Mag_AnalyticSignal_cog.tif to log...
